In [1]:
import numpy as np

In [2]:
def load_BV_binary_vectors(path, resid, phmin, phmax):
    x=resid
    p00_list = []
    p10_list = []
    p01_list = []
    p11_list = []
    m_min = phmin
    m_max = phmax
    m_values = np.arange(m_min, m_max + 0.1, 0.5)
    for m in m_values:
        p00_list.append(np.loadtxt(f'{path}/{m:.1f}/p00_res{x}_pH{m:.1f}.dat'))
        p10_list.append(np.loadtxt(f'{path}/{m:.1f}/p10_res{x}_pH{m:.1f}.dat'))
        p01_list.append(np.loadtxt(f'{path}/{m:.1f}/p01_res{x}_pH{m:.1f}.dat'))
        p11_list.append(np.loadtxt(f'{path}/{m:.1f}/p11_res{x}_pH{m:.1f}.dat'))
    return np.stack(p00_list, axis=-1), np.stack(p10_list, axis=-1), np.stack(p01_list, axis=-1), np.stack(p11_list, axis=-1)

def load_residue_binary_vectors(path, resid):
    temp_list = []
    m_min = 3.5
    m_max = 10.0
    m_values = np.arange(m_min, m_max + 0.1, 0.5)
    for m in m_values:
        temp_list.append(np.loadtxt(f'{path}/binary_vectors_{m:.1f}/res{resid}_pH{m:.1f}.dat'))
    return np.stack(temp_list, axis=-1)

def mean_value(array):
    mean_values_list = []
    for ph in range(0,14,1):
        mean_values_list.append(np.mean(array[:,ph]))
    return np.stack(mean_values_list)


def bootstrap_with_replacement(data, num_samples):
    data = np.array(data) 
    n = len(data)
    prot_frac = np.zeros(num_samples)

    for i in range(num_samples):
        new_sample = np.random.choice(data, size=n, replace=True) # Simple bootstrapping with replacement
        prot_frac[i] = np.mean(new_sample) # The protonation fraction is the arithmetic mean of the time series

    mean_of_prot_fracs = np.mean(prot_frac)
    std_of_prot_fracs = np.std(prot_frac)

    return mean_of_prot_fracs, std_of_prot_fracs


def autocorrelation(data, max_lag):
    mean = np.mean(data)
    data_demeaned = data - mean
    autocorr_values = []
    n = len(data)

    for lag in range(max_lag + 1):
        if lag < n:
            shifted_data = data[lag:]
            std1 = np.std(shifted_data)
            shifted_data = shifted_data - np.mean(shifted_data)
            original_data = data[:n - lag]
            std2 = np.std(original_data)
            original_data = original_data - np.mean(original_data)
            
            covariance = np.mean(original_data * shifted_data)
            autocorr_values.append(covariance / (std1*std2))
    return np.array(autocorr_values)

def crosscorrelation(data1, data2, max_lag):
    mean1 = np.mean(data1)
    mean2 = np.mean(data2)
    data_demeaned1 = data1 - mean1
    data_demeaned2 = data2 - mean2
    crosscorr_values = []
    n = len(data1)

    for lag in range(max_lag + 1):
        if lag < n: 
            if lag > 0:
                shifted_data = data_demeaned1[lag:]
                original_data = data_demeaned2[:n - lag]
            else:
                shifted_data = data_demeaned1
                original_data = data_demeaned2

            covariance = np.mean(original_data * shifted_data)
            std1 = np.std(shifted_data)
            std2 = np.std(original_data)
            crosscorr_values.append(covariance / (std1 * std2))
    return np.array(crosscorr_values)

In [3]:
path_free_bv = './state_vectors/chromophore/free-BV_full_records/'
path_mon_bv = './state_vectors/monomer/BV_binary_vectors/'
path_tet_bv = './state_vectors/tetramer/BV_binary_vectors/'

x00, x10, x01, x11 = load_BV_binary_vectors(path_free_bv, 0, 1.0, 9.5)
x00m, x10m, x01m, x11m = load_BV_binary_vectors(path_mon_bv, 27, 3.5, 10.0)

x00a, x10a, x01a, x11a = load_BV_binary_vectors(path_tet_bv, 54, 3.5, 10.0)
x00b, x10b, x01b, x11b = load_BV_binary_vectors(path_tet_bv, 55, 3.5, 10.0)
x00c, x10c, x01c, x11c = load_BV_binary_vectors(path_tet_bv, 110, 3.5, 10.0)
x00d, x10d, x01d, x11d = load_BV_binary_vectors(path_tet_bv, 111, 3.5, 10.0)

x00t = (x00a+x00b+x00c+x00d)/4
x01t = (x01a+x01b+x01c+x01d)/4
x10t = (x10a+x10b+x10c+x10d)/4
x11t = (x11a+x11b+x11c+x11d)/4

In [4]:
x00_list = []
x01_list = []
x10_list = []
x11_list = []
for i in range(0,18,1):
    _, eb00 = bootstrap_with_replacement(x00[:,i], 1000)
    _, eb01 = bootstrap_with_replacement(x01[:,i], 1000)
    _, eb10 = bootstrap_with_replacement(x10[:,i], 1000)
    _, eb11 = bootstrap_with_replacement(x11[:,i], 1000)
    x00_list.append(eb00)
    x01_list.append(eb01)
    x10_list.append(eb10)
    x11_list.append(eb11)

In [5]:
print(
    f"eb00 = {x00_list}\n"
    f"eb01 = {x01_list}\n"
    f"eb10 = {x10_list}\n"
    f"eb11 = {x11_list}"
)

eb00 = [0.0, 0.0, 0.0, 0.0, 0.000335222066859328, 0.0007265627406066583, 0.0017651390386249199, 0.0033649779837917147, 0.0047410955503782645, 0.005011511374784619, 0.003750303541584523, 0.0024237462206481365, 0.0013694943841692801, 0.0007870991353031664, 0.0004200783439697009, 0.00030913570817221736, 0.0, 0.0]
eb01 = [0.0002049769657023704, 0.0005084874094068608, 0.0007704411970660553, 0.0012700232210766838, 0.0022513433833843623, 0.0035044654344979493, 0.004298068070358893, 0.004446328916548434, 0.0046045376732681215, 0.003635154048849206, 0.002504687302951868, 0.0016478621280091724, 0.0009696154906758174, 0.0005617711184158855, 0.0002849788528971068, 0.00019870516565150302, 0.0, 0.0]
eb10 = [0.00022339037322408853, 0.0004522410127001609, 0.0008523469561559501, 0.0014587124632056577, 0.0024474133311925346, 0.0036246842016242386, 0.004611050855215569, 0.004754224895090979, 0.00466445288841887, 0.003993344367905663, 0.0027377018250337873, 0.001694484451323195, 0.0009520403115337267, 0.0

In [63]:
x00m_list = []
x01m_list = []
x10m_list = []
x11m_list = []
for i in range(0,14,1):
    _, eb00m = bootstrap_with_replacement(x00m[:,i], 1000)
    _, eb01m = bootstrap_with_replacement(x01m[:,i], 1000)
    _, eb10m = bootstrap_with_replacement(x10m[:,i], 1000)
    _, eb11m = bootstrap_with_replacement(x11m[:,i], 1000)
    x00m_list.append(eb00m)
    x01m_list.append(eb01m)
    x10m_list.append(eb10m)
    x11m_list.append(eb11m)

In [72]:
print(
    f"eb00m = {x00m_list}\n"
    f"eb01m = {x01m_list}\n"
    f"eb10m = {x10m_list}\n"
    f"eb11m = {x11m_list}"
)

eb00m = [0.0008504112171580141, 0.001338913986763854, 0.001862627126760806, 0.002645520607002494, 0.0033458353247898206, 0.0034645880809173464, 0.003234509007406376, 0.0026131765901368305, 0.002030854389224903, 0.0013739070358749265, 0.0009567091033751701, 0.0006718623046992981, 0.0005388409429673531, 0.00030379714942453396]
eb01m = [0.003461912532647169, 0.003664612376232954, 0.003447516153845182, 0.003564740101470577, 0.0033297698438571207, 0.0029910936250017394, 0.002374136778571182, 0.0016667126859292302, 0.0011671216975058295, 0.0007790974243975189, 0.0004924137917032744, 0.00035493534520405926, 0.0002941584383712127, 0.00022978054194873948]
eb10m = [0.0035897779311155533, 0.0034149476367301987, 0.0034375111286631243, 0.0035196032455136754, 0.003309994571551963, 0.0030155740744617645, 0.002549384248254287, 0.0020536022061866483, 0.0016292781641358907, 0.0011216640590339612, 0.0007936107462879976, 0.0005556655471666915, 0.0004460079227950696, 0.00021815093604643466]
eb11m = [0.0022

In [62]:
x00t_list = []
x01t_list = []
x10t_list = []
x11t_list = []
for i in range(0,14,1):
    _, eb00t = bootstrap_with_replacement(x00t[:,i], 1000)
    _, eb01t = bootstrap_with_replacement(x01t[:,i], 1000)
    _, eb10t = bootstrap_with_replacement(x10t[:,i], 1000)
    _, eb11t = bootstrap_with_replacement(x11t[:,i], 1000)
    x00t_list.append(eb00t)
    x01t_list.append(eb01t)
    x10t_list.append(eb10t)
    x11t_list.append(eb11t)

In [73]:
print(
    f"eb00t = {x00t_list}\n"
    f"eb01t = {x01t_list}\n"
    f"eb10t = {x10t_list}\n"
    f"eb11t = {x11t_list}"
)

eb00t = [0.00018884749380651008, 0.00023163775439451523, 0.00033704892725212594, 0.0005655128688312554, 0.0007706093706510383, 0.001082980958589399, 0.0013658129138679896, 0.0016587694304495335, 0.001742399420463006, 0.001637993552644391, 0.0015257192153679648, 0.0012366544968527, 0.0009769469336543151, 0.0006351704325905799]
eb01t = [0.0009677792687318784, 0.0011106573857104886, 0.0013158997772597996, 0.0012443490373565053, 0.0011695177234671437, 0.0012128275381737658, 0.001242444271067282, 0.0011266707900772792, 0.0010660179572739338, 0.0009947499232396066, 0.0008832080331987864, 0.0007514400323511334, 0.0005465543970706849, 0.0003183082705098033]
eb10t = [0.0013739556965014453, 0.0015673908402333344, 0.001426735644513772, 0.0013786778419427553, 0.001363243359401387, 0.0014430155639307029, 0.001577313155953746, 0.0017017035525044317, 0.0016358641105455713, 0.0015839208426257976, 0.001341555393286836, 0.001119937231284816, 0.0008788076183841261, 0.0005437454328038377]
eb11t = [0.00118

In [6]:
X = x01 + x10 + 2*x11
Xm = x01m + x10m + 2*x11m
Xt = x01t + x10t + 2*x11t

X_list = []
for i in range(0,18,1):
    _, ebX = bootstrap_with_replacement(X[:,i], 1000)
    X_list.append(ebX)

Xm_list = []
for i in range(0,14,1):
    _, ebXm = bootstrap_with_replacement(Xm[:,i], 1000)
    Xm_list.append(ebXm)

Xt_list = []
for i in range(0,14,1):
    _, ebXt = bootstrap_with_replacement(Xt[:,i], 1000)
    Xt_list.append(ebXt)

In [7]:
print(
    f"ebX = {X_list}\n"
    f"ebXm = {Xm_list}\n"
    f"ebXt = {Xt_list}"
)

ebX = [0.0002982084060854431, 0.0006684392816982163, 0.0010944936750570526, 0.0019233382715961036, 0.0033268609887568564, 0.004789345462026177, 0.005532762329260766, 0.005166393244735823, 0.005491837546074237, 0.004915500899049944, 0.0035933378376540297, 0.0022855543717018004, 0.0013283983039520907, 0.0008054309155482339, 0.0003980836775006423, 0.0003039098005050507, 0.0, 0.0]
ebXm = [0.0022973813768475677, 0.002071374982913893, 0.0020480728427662346, 0.002700146813136606, 0.0033463139572215013, 0.0035780888437147353, 0.0031921789504004696, 0.002536923290805559, 0.001952190481409232, 0.0013414597353551304, 0.0009475462856024787, 0.000661161882902835, 0.0005386758692436113, 0.00032012265132697884]
ebXt = [0.001231454768441731, 0.0010789216027623726, 0.0009308002716987369, 0.0008372310027987539, 0.000891615713885854, 0.001124153458047865, 0.0013576652631522228, 0.0016568527513129046, 0.0017005166907921808, 0.001681243576874221, 0.00149112688377046, 0.0012302826500523603, 0.00097249480970

In [8]:
print(np.min(X_list), np.max(X_list))
print(np.min(Xm_list), np.max(Xm_list))
print(np.min(Xt_list), np.max(Xt_list))

0.0 0.005532762329260766
0.00032012265132697884 0.0035780888437147353
0.0006291273466188467 0.0017005166907921808


In [9]:
apoprotein_idx = np.arange(0,27,1)
path_apo = './state_vectors/apoprotein/apoprotein_all_residues/'
apo_error_list = [] 

for idx in apoprotein_idx:
    n = load_residue_binary_vectors(path_apo, idx)

    temp_list = []
    for i in range(0, 14):
        _, a = bootstrap_with_replacement(n[:, i], 1000) 
        temp_list.append(a)

    apo_error_list.append(np.array(temp_list)) 

apo_errors = np.array(apo_error_list)

In [10]:
apo_errors.shape

(27, 14)

In [11]:
apo_residues = [
    "LYS23", "LYS29", "GLU34", "ASP35", "ASP37", "ASP47", "LYS54", "LYS57", "GLU59", "GLU79", "LYS87", "GLU96", 
    "ASP97", "GLU100", "LYS103", "HIS108", "GLU109", "ASP122", "ASP124", "HIS139", "GLU135", "GLU136", "LYS158", 
    "LYS159", "ASP157", "ASP167", "TYR183"
]

for i, row in enumerate(apo_errors):
    print(f"{apo_residues[i]} ", end="") 
    print(" ".join(f"{value:.4f}" for value in row)) 

LYS23 0.0000 0.0000 0.0000 0.0001 0.0001 0.0001 0.0002 0.0003 0.0003 0.0005 0.0010 0.0016 0.0025 0.0032
LYS29 0.0001 0.0000 0.0001 0.0002 0.0002 0.0003 0.0004 0.0005 0.0006 0.0009 0.0013 0.0019 0.0028 0.0034
GLU34 0.0024 0.0029 0.0033 0.0036 0.0033 0.0027 0.0018 0.0012 0.0007 0.0005 0.0003 0.0002 0.0002 0.0001
ASP35 0.0021 0.0029 0.0032 0.0035 0.0031 0.0024 0.0016 0.0009 0.0005 0.0004 0.0002 0.0001 0.0000 0.0000
ASP37 0.0034 0.0034 0.0029 0.0021 0.0015 0.0009 0.0006 0.0003 0.0002 0.0001 0.0000 0.0000 0.0000 0.0000
ASP47 0.0036 0.0031 0.0023 0.0015 0.0010 0.0007 0.0005 0.0004 0.0003 0.0002 0.0001 0.0000 0.0000 0.0000
LYS54 0.0009 0.0008 0.0010 0.0016 0.0022 0.0026 0.0034 0.0037 0.0033 0.0025 0.0017 0.0012 0.0009 0.0006
LYS57 0.0000 0.0001 0.0001 0.0001 0.0002 0.0002 0.0003 0.0005 0.0007 0.0010 0.0017 0.0026 0.0034 0.0035
GLU59 0.0011 0.0008 0.0004 0.0001 0.0001 0.0001 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000
GLU79 0.0021 0.0028 0.0034 0.0034 0.0025 0.0020 0.0014 0.0009 0.

In [14]:
apo_residues = [
    "LYS23", "LYS29", "GLU34", "ASP35", "ASP37", "ASP47", "LYS54", "LYS57", "GLU59", "GLU79", "LYS87", "GLU96", 
    "ASP97", "GLU100", "LYS103", "HIS108", "GLU109", "ASP122", "ASP124", "HIS139", "GLU135", "GLU136", "LYS158", 
    "LYS159", "ASP157", "ASP167", "TYR183"
]

# Generate the pH range from 3.5 to 10 with a step of 0.5
pH_values = [round(x, 1) for x in np.arange(3.5, 10.5, 0.5)]

# Print table header
header = "pH\t" + "\t".join(f"{pH}" for pH in pH_values)
print(header)

# Print each row
for i, row in enumerate(apo_errors):
    row_data = "\t".join(f"{value:.4f}" for value in row)
    print(f"{apo_residues[i]}\t{row_data}")


pH	3.5	4.0	4.5	5.0	5.5	6.0	6.5	7.0	7.5	8.0	8.5	9.0	9.5	10.0
LYS23	0.0000	0.0000	0.0000	0.0001	0.0001	0.0001	0.0002	0.0003	0.0003	0.0005	0.0010	0.0016	0.0025	0.0032
LYS29	0.0001	0.0000	0.0001	0.0002	0.0002	0.0003	0.0004	0.0005	0.0006	0.0009	0.0013	0.0019	0.0028	0.0034
GLU34	0.0024	0.0029	0.0033	0.0036	0.0033	0.0027	0.0018	0.0012	0.0007	0.0005	0.0003	0.0002	0.0002	0.0001
ASP35	0.0021	0.0029	0.0032	0.0035	0.0031	0.0024	0.0016	0.0009	0.0005	0.0004	0.0002	0.0001	0.0000	0.0000
ASP37	0.0034	0.0034	0.0029	0.0021	0.0015	0.0009	0.0006	0.0003	0.0002	0.0001	0.0000	0.0000	0.0000	0.0000
ASP47	0.0036	0.0031	0.0023	0.0015	0.0010	0.0007	0.0005	0.0004	0.0003	0.0002	0.0001	0.0000	0.0000	0.0000
LYS54	0.0009	0.0008	0.0010	0.0016	0.0022	0.0026	0.0034	0.0037	0.0033	0.0025	0.0017	0.0012	0.0009	0.0006
LYS57	0.0000	0.0001	0.0001	0.0001	0.0002	0.0002	0.0003	0.0005	0.0007	0.0010	0.0017	0.0026	0.0034	0.0035
GLU59	0.0011	0.0008	0.0004	0.0001	0.0001	0.0001	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000
GLU7

In [15]:
monomer_idx = np.arange(0,27,1)
path_mon = './state_vectors/monomer/monomer_all_residues/'
mon_error_list = [] 

for idx in monomer_idx:
    n = load_residue_binary_vectors(path_mon, idx)

    temp_list = []
    for i in range(0, 14):
        _, a = bootstrap_with_replacement(n[:, i], 1000) 
        temp_list.append(a)

    mon_error_list.append(np.array(temp_list)) 

mon_residue_errors = np.array(mon_error_list)

In [16]:
mon_residue_errors.shape

(27, 14)

In [24]:
mon_residues = [
    "LYS23", "LYS29", "GLU34", "ASP35", "ASP37", "ASP47", "LYS54", "LYS57", "GLU59", "GLU79", "LYS87", "GLU96", 
    "ASP97", "GLU100", "LYS103", "HIS108", "GLU109", "ASP122", "ASP124", "HIS139", "GLU135", "GLU136", "LYS158", 
    "LYS159", "ASP157", "ASP167", "TYR183"
]

# Generate the pH range from 3.5 to 10 with a step of 0.5
pH_values = [round(x, 1) for x in np.arange(3.5, 10.5, 0.5)]

# Print table header
header = "pH\t" + "\t".join(f"{pH}" for pH in pH_values)
print(header)

# Print each row
for i, row in enumerate(mon_residue_errors):
    row_data = "\t".join(f"{value:.4f}" for value in row)
    print(f"{mon_residues[i]}\t{row_data}")


pH	3.5	4.0	4.5	5.0	5.5	6.0	6.5	7.0	7.5	8.0	8.5	9.0	9.5	10.0
LYS23	0.0000	0.0001	0.0000	0.0001	0.0001	0.0002	0.0003	0.0004	0.0004	0.0008	0.0011	0.0017	0.0026	0.0034
LYS29	0.0001	0.0002	0.0001	0.0002	0.0003	0.0003	0.0004	0.0005	0.0008	0.0012	0.0017	0.0024	0.0030	0.0034
GLU34	0.0024	0.0031	0.0034	0.0035	0.0030	0.0023	0.0017	0.0010	0.0006	0.0003	0.0002	0.0002	0.0001	0.0001
ASP35	0.0019	0.0027	0.0033	0.0036	0.0031	0.0023	0.0015	0.0009	0.0005	0.0003	0.0002	0.0001	0.0000	0.0000
ASP37	0.0034	0.0033	0.0027	0.0020	0.0014	0.0009	0.0005	0.0003	0.0002	0.0001	0.0000	0.0001	0.0000	0.0001
ASP47	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000
LYS54	0.0000	0.0000	0.0000	0.0000	0.0001	0.0001	0.0002	0.0004	0.0005	0.0008	0.0014	0.0022	0.0030	0.0035
LYS57	0.0001	0.0002	0.0003	0.0004	0.0004	0.0004	0.0005	0.0006	0.0009	0.0013	0.0019	0.0027	0.0033	0.0035
GLU59	0.0016	0.0024	0.0034	0.0035	0.0029	0.0018	0.0011	0.0006	0.0004	0.0002	0.0001	0.0001	0.0000	0.0000
GLU7

In [18]:
tetramer_idx = np.arange(0,110,1)
tetramer_idx = np.delete(tetramer_idx, [54, 55]) # remove biliverdin indexes
path_tet = './state_vectors/tetramer/tetramer_all_residues/'
tet_error_list = [] 

for idx in tetramer_idx:
    n = load_residue_binary_vectors(path_tet, idx)

    temp_list = []
    for i in range(0, 14):
        _, a = bootstrap_with_replacement(n[:, i], 1000) 
        temp_list.append(a)

    tet_error_list.append(np.array(temp_list)) 

tet_residue_errors = np.array(tet_error_list)

In [22]:
tet_residue_errors[72]

array([3.57799869e-03, 3.45765592e-03, 3.07460141e-03, 2.20671181e-03,
       1.44200984e-03, 9.47909659e-04, 6.69143620e-04, 3.80473236e-04,
       2.27915122e-04, 8.75506195e-05, 7.06706485e-05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

In [25]:
tet_residues = [
    "LYS23", "LYS29", "GLU34", "ASP35", "ASP37", "ASP47",
    "LYS54", "LYS57", "GLU59", "GLU79", "LYS85", "GLU96",
    "ASP97", "GLU100", "LYS103", "HIS106", "GLU107", "ASP120",
    "ASP122", "HIS137", "GLU154", "GLU155", "LYS156", "LYS157",
    "ASP172", "ASP180", "TYR183", "LYS192", "LYS198", "GLU203",
    "ASP204", "ASP206", "ASP216", "LYS223", "LYS226", "GLU228",
    "GLU248", "LYS256", "GLU265", "ASP266", "GLU269", "LYS272",
    "HIS277", "GLU278", "ASP291", "ASP293", "HIS308", "GLU323",
    "GLU324", "LYS327", "LYS328", "ASP341", "ASP349", "TYR352",
    "LYS363", "LYS369", "GLU374", "ASP375",
    "ASP377", "ASP387", "LYS394", "LYS397", "GLU400", "GLU419",
    "LYS427", "GLU436", "ASP437", "GLU440", "LYS443", "HIS448",
    "GLU449", "ASP462", "ASP464", "HIS479", "GLU494", "GLU495",
    "LYS498", "LYS499", "ASP520", "ASP528", "TYR532", "LYS538",
    "LYS539", "GLU543", "ASP544", "ASP546", "ASP556", "LYS563",
    "LYS566", "GLU568", "GLU588", "LYS596", "GLU605", "ASP606",
    "GLU609", "LYS612", "HIS617", "GLU618", "ASP631", "ASP633",
    "HIS648", "GLU663", "GLU664", "LYS667", "LYS668", "ASP681",
    "ASP689", "TYR692", "LYS703", "LYS709",
    "GLU714", "ASP715", "ASP717", "ASP727", "LYS734", "LYS737"
]

# Generate the pH range from 3.5 to 10 with a step of 0.5
pH_values = [round(x, 1) for x in np.arange(3.5, 10.5, 0.5)]

# Print table header
header = "pH\t" + "\t".join(f"{pH}" for pH in pH_values)
print(header)

# Print each row
for i, row in enumerate(tet_residue_errors):
    row_data = "\t".join(f"{value:.4f}" for value in row)
    print(f"{tet_residues[i]}\t{row_data}")


pH	3.5	4.0	4.5	5.0	5.5	6.0	6.5	7.0	7.5	8.0	8.5	9.0	9.5	10.0
LYS23	0.0001	0.0004	0.0003	0.0001	0.0003	0.0004	0.0004	0.0006	0.0009	0.0011	0.0016	0.0020	0.0027	0.0032
LYS29	0.0005	0.0006	0.0008	0.0007	0.0009	0.0010	0.0010	0.0011	0.0013	0.0018	0.0022	0.0029	0.0033	0.0034
GLU34	0.0026	0.0033	0.0036	0.0035	0.0030	0.0024	0.0021	0.0012	0.0006	0.0004	0.0003	0.0003	0.0002	0.0000
ASP35	0.0023	0.0030	0.0033	0.0035	0.0030	0.0022	0.0015	0.0010	0.0006	0.0004	0.0002	0.0001	0.0001	0.0000
ASP37	0.0035	0.0034	0.0031	0.0022	0.0015	0.0010	0.0007	0.0004	0.0003	0.0002	0.0001	0.0000	0.0000	0.0000
ASP47	0.0000	0.0000	0.0001	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000
LYS54	0.0002	0.0000	0.0001	0.0002	0.0003	0.0004	0.0008	0.0011	0.0015	0.0019	0.0027	0.0034	0.0034	0.0029
LYS57	0.0002	0.0005	0.0004	0.0004	0.0007	0.0010	0.0013	0.0014	0.0018	0.0019	0.0023	0.0027	0.0032	0.0036
GLU59	0.0033	0.0035	0.0035	0.0032	0.0024	0.0017	0.0011	0.0008	0.0006	0.0003	0.0003	0.0002	0.0001	0.0001
GLU7